# Calculating bne utilities with high precision

We usually use an evaluation batch size of `2**22`, but in some settings, we desire more accuracy.
This notebook calculates high-precision bne utilities in the LLG settings.

It uses an outer loop of `2**8` iterations of evaluating the utility with a batch_size of `2**26`, giving a total `2**34` MC samples.

Results:

In [ ]:
nearest_zero = [0.13399262726306915, 0.13399307429790497, 0.46403446793556213]
nearest_bid = [0.12500184774398804, 0.12499602138996124, 0.49999746680259705]
nearest_vcg = [0.13316573202610016, 0.13316750526428223, 0.4673408269882202]

In [ ]:
payment_rule = 'nearest_vcg'

In [ ]:
import torch
from tqdm import tqdm

import os
import sys
root_path = os.path.join(os.path.expanduser('~'), 'bnelearn')
if root_path not in sys.path:
    sys.path.append(root_path)
    
from bnelearn.experiment import LLGExperiment, presets
from bnelearn.experiment import LearningConfiguration
from bnelearn.experiment.gpu_controller import GPUController

In [ ]:
running_configuration, logging_configuration, experiment_configuration, experiment_class =\
        presets.llg(1,1,payment_rule,specific_gpu=1, logging=False)
experiment_configuration.n_players = 3

logging_configuration.eval_batch_size = 2**26
gpu_configuration = GPUController(specific_gpu=7)

learning_configuration = LearningConfiguration(
        pretrain_iters=1,
        batch_size=2**2,
        learner_hyperparams = {'population_size': 3,
                               'sigma': 1.,
                               'scale_sigma_by_model_size': True})
experiment = experiment_class(experiment_configuration, learning_configuration,
                                          logging_configuration, gpu_configuration)

In [ ]:
experiment._setup_eval_environment()

In [ ]:
utilities = []

sys.stdout = open(os.devnull, 'w')

for _ in tqdm(range(2**8)):
    experiment._setup_eval_environment()
    utilities.append(experiment.bne_utilities.unsqueeze(0))    
    
sys.stdout = sys.__stdout__

In [ ]:
mean_utils = torch.cat(utilities, dim=0).mean(dim=0)
result = [i.item() for i in mean_utils]
result

In [ ]:
del experiment
torch.cuda.empty_cache()
torch.cuda.ipc_collect()